# Train Logistic Regression classifier with Apache SparkML


In [ ]:
!pip install pyspark==2.4 pyspark2pmml==0.5.1 #mleap==0.15

In [ ]:
!wget https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/jpmml-sparkml-executable-1.5.12.jar

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark2pmml import PMMLBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler

In [ ]:
data_parquet = os.environ.get('data_parquet',
                              'data.parquet')  # input file name (parquet)
master = os.environ.get('master',
                        "local[*]")  # URL to Spark master
model_target = os.environ.get('model_target',
                              "model.xml")  # model output file name
data_dir = os.environ.get('data_dir',
                          '../../data/')  # temporary directory for data
input_columns = os.environ.get('input_columns',
                               '["x", "y", "z"]')  # input columns to consider

In [ ]:
sc = SparkContext.getOrCreate(
    SparkConf().
    setMaster(master).
    set("spark.jars", 'jpmml-sparkml-executable-1.5.12.jar')
)

spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
df = spark.read.parquet(data_dir + data_parquet)

In [ ]:
# register a corresponding query table
df.createOrReplaceTempView('df')

In [ ]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [ ]:
indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=eval(input_columns),
                                  outputCol="features")

normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [ ]:
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr])

In [ ]:
model = pipeline.fit(df_train)

In [ ]:
prediction = model.transform(df_train)

In [ ]:
binEval = MulticlassClassificationEvaluator(). \
    setMetricName("accuracy"). \
    setPredictionCol("prediction"). \
    setLabelCol("label")

binEval.evaluate(prediction)

In [ ]:
pmmlBuilder = PMMLBuilder(sc, df_train, model)
pmmlBuilder.buildFile(data_dir + model_target)